In [23]:
import warnings
warnings.filterwarnings('ignore')

MY_API_TOKEN = ""
TEST_GRACEID = "MS181101ab"

### The Treasure Map API: Pointings
The motivation for the Pointings interface in the TreasureMap API is for Astronomers who search for GW counterparts to share their observation plans in a transparent manner to optimize search efforts in real time. The goal is for the Treasure Map to serve as a central repository for all Planned and Completed schedules. User's will be able to post their search plans, and query other team's search plans with this interface. In doing so, the community can work hand in hand to cover the GW localization region in an optimal manner by covering regions or depths that haven't been reported yet.

### Pointing model
The `gwtm_api.Pointing` model covers the absolute basic parameters of a photometric astronomical observation. But most importantly it requires an instrument footprint to already be established in the Treasure Map. This can be accomplished by having a valid account, and going to the [Submit Instrument](https://treasuremap.space/submit_instrument) page, and following the instructions there. 

The `gwtm_api.Pointing` fields are as follows:

- `ra: float` - the Right Ascenscion (RA) of the pointing.
- `dec: float` - the Declination (DEC) of the pointing.
  - The `ra` and `dec` values can also be input as `position: str` that has the strict format of `"POINT (ra dec)"`.
- `instrumentid: int` - the instrument id recieved when an instrument is submitted
- `time: str | datetime` - the UTC time associated with the pointing. The string format of the time `%Y-%m-%dT%H:%M:%S.%f` format. e.g. `"2019-05-01T12:00:00.00"`. Otherwise `datetime.datetime` will be converted to that format. 
- `status : str | gwtm_api.enums.pointing_status` - the status of the pointing observation. Valid inputs: `completed`, `planned`.
- `depth: float` - the limiting magnitude/brightness of the pointing. 
- `depth_unit: str | gwtm_api.enums.depth_unit` - units for depth. Can be: `ab_mag`, `vega_ma"`, `flux_erg`, or `flux_jy`.
- `pos_angle: float` - the position angle of the telescope exposure (not required for `pointing.status = planned`)
- `depth_err: float` - the error in the depth, can be a rough estimate (not required for `pointing.status = planned`)
- **Bandpass Information** can be passed in by stating the band, or the range of wavelengths, energies, or frequencies the pointing covers. If your pointing is associated with a bandpass, we will calculate the wavelength range from a generic set of filters from [SVO](http://svo2.cab.inta-csic.es/theory/fps/index.php?asttype=astro)
  - *Method 1*: Bandpass list
    - `band: str | gwtm_api.enums.bandpass` - the name of the bandpass. Can be one of: `U`, `B`, `V`, `R`, `I`, `J`, `H`, `K`, `u`, `g`, `r`, `i`, `z`, `UVW1`, `UVW2`, `XRT`, `clear`, `open`, `UHF`, `VHF`, `L`, `S`, `C`, `X`, `other`, `TESS`, `BAT`, `HESS`, `WISEL`, and `q`
  - *Method 2*: Central Wavelength and Bandwidth
    - `central_wave: float` - the central wavelength in Angstroms
    - `bandwidth: float` - the bandwidth in Angstroms
  - *Method 3*: Spectrum Regimes: There are 3 ways to submit a spectrum regime for your associated pointing. By stating the wavelength, energy, or frequency regimes along with their associated units
    - `wavelength_regime: list[float]` - the begin and endpoints of the wavelength bandwidth e.g. `[low, high]`
    - `wavelength_unit: str | gwtm_api.enums.wavelength_units` - valid inputs are `angstrom`, `nanometer` or `micron`
    - or `energy_regime: list[float]` - the begin and endpoints of the energy bandwidth e.g. `[low, high]`
    - with `energy_unit: str | gwtm_api.enums.energy_units` - valid inputs are: `eV`, `keV`, `MeV`, `GeV`, and `TeV`
    - or `frequency_regime: list[float]` - the begin and endpoints of the frequency bandwidth e.g, `[low, high]`
    - with `frequench_unit: str | gwtm_api.enums.frequency_units` - valid inputs are: `Hz`, `kHz`, `MHz`, `GHz` and `THz`

### Using the gwtm_api.Pointing api_wrapper.
##### Instantiating a single Pointing and posting

In [24]:
import datetime
from gwtm_api import Pointing
from gwtm_api import enums as gwenums

#I am going to create a pointing object corresponding to an observation of the 
# AT2017gfo with my instrument
my_pointing = Pointing(
    ra=197.4503,
    dec=-23.3815,
    instrumentid=10, #DLT40
    time=datetime.datetime(2017, 8, 18, 1, 31),
    depth=18.5,
    band=gwenums.bandpass.open,
    depth_unit=gwenums.depth_unit.ab_mag,
    pos_angle=0.0,
    status=gwenums.pointing_status.completed
)
my_pointing.post(api_token=MY_API_TOKEN, graceid=TEST_GRACEID, verbose=True)
my_pointing.dump()

{'pointing_ids': [47470], 'ERRORS': [], 'WARNINGS': []}
position: POINT (-162.5497 -23.3815)
time: 2017-08-18 01:31:00
ra: -162.5497
dec: -23.3815
instrumentid: 10
status: completed
depth: 18.5
depth_unit: ab_mag
band: open
pos_angle: 0.0
id: 47470
depth_err: None
submitterid: 2
doi_url: 
doi_id: None
central_wave: 7000.0
bandwidth: 3000.0



##### Batch Pointing Submission
Here we will give examples of instantiating multiple Pointing objects and posting the list in a single request

In [25]:
#create a list of valid pointings

batch = [
    Pointing(
        position="POINT (257.0 -12.0)",
        instrumentid=47, #ZTF
        time="2019-11-11T12:20:51.00",
        pos_angle=0.0,
        band="open",
        depth=21.5,
        depth_unit="ab_mag",
        status="completed"
    ),
    Pointing(
        ra=257,
        dec=-27,
        instrumentid=38,
        depth=22,
        central_wave=7000,
        bandwidth=3000,
        depth_unit=gwenums.depth_unit.ab_mag,
        status=gwenums.pointing_status.completed,
        pos_angle=0.0,
        time=datetime.datetime.now()
    ),
    Pointing(
        ra=257,
        dec=-27,
        instrumentid=13,
        energy_regime=[0.1,10],
        energy_unit=gwenums.energy_units.keV,
        time="2019-11-11T13:25:55.00Z",
        status=gwenums.pointing_status.planned,
        depth=1.6e-12,
        depth_unit=gwenums.depth_unit.flux_erg 
    )
]

batch = Pointing.batch_post(api_token=MY_API_TOKEN, graceid=TEST_GRACEID, pointings=batch, verbose=True)

{'pointing_ids': [47471, 47472, 47473], 'ERRORS': [], 'WARNINGS': []}


In [26]:
for b in batch:
    b.dump()

position: POINT (-103 -12)
time: 2019-11-11 12:20:51
id: 47471
status: completed
instrumentid: 47
depth: 21.5
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 0.0
band: open
doi_url: 
doi_id: None
central_wave: 7000.0
bandwidth: 3000.0
ra: -103.0
dec: -12.0

position: POINT (-103 -27)
time: 2024-05-10 14:31:04.866623
id: 47472
status: completed
instrumentid: 38
depth: 22.0
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 0.0
band: open
doi_url: 
doi_id: None
central_wave: 7000.0
bandwidth: 3000.0
ra: -103.0
dec: -27.0

position: POINT (-103 -27)
time: 2019-11-11 13:25:55
id: 47473
status: planned
instrumentid: 13
depth: 1.6e-12
depth_err: None
depth_unit: flux_erg
submitterid: 2
pos_angle: None
band: BAT
doi_url: 
doi_id: None
central_wave: 1.2398
bandwidth: 0.0
ra: -103.0
dec: -27.0



#### Querying for submitted pointings
Here we will give examples for utilizing the `gwtm_api.Pointing.get()` method to search for existing pointings.

In [27]:
queried_pointings = Pointing.get(
    api_token=MY_API_TOKEN, graceid=TEST_GRACEID
)
for q in queried_pointings:
    q.dump()

position: POINT (15 -24)
time: 2024-05-07 14:35:27.471849
id: 47440
status: completed
instrumentid: 10
depth: 18.5
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 50.0
band: r
doi_url: 
doi_id: None
central_wave: 6415.4
bandwidth: 1487.58
ra: 15.0
dec: -24.0

position: POINT (15 -24)
time: 2024-05-07 16:57:00.602272
id: 47443
status: completed
instrumentid: 10
depth: 18.5
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 50.0
band: r
doi_url: 
doi_id: None
central_wave: 6415.4
bandwidth: 1487.58
ra: 15.0
dec: -24.0

position: POINT (15 -24)
time: 2024-05-07 14:56:21.184701
id: 47441
status: completed
instrumentid: 10
depth: 18.5
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 50.0
band: r
doi_url: 
doi_id: None
central_wave: 6415.4
bandwidth: 1487.58
ra: 15.0
dec: -24.0

position: POINT (18 -30)
time: 2024-05-07 13:56:21.184862
id: 47442
status: completed
instrumentid: 10
depth: 18.5
depth_err: None
depth_unit: ab_mag
submitterid: 2
pos_angle: 50.0


##### Further filtering options
- `instrument: str` - name or id of instrument
- `instruments: list[str]` - list of ids or names for instruments
- `id: int` - pointing id
- `ids: list[int]` - list of pointing ids
- `status: str | gwtm_api.enums.pointing_status` - pointing status
- `completed_after: str | datetime.datetime` - pointings with a `completed` status and time after
- `completed_before: str | datetime.datetime` - pointings with a `completed` status and time before
- `planned_after: str | datetime.datetime` - pointings with a `planned` status and time after
- `planned_before: str | datetime.datetime` - pointings with a `planned` status and time before
- `band: str | gwtm_api.enums.bandpass` - pointings associated to a bandpass
- `bands: list[str] | list[gwtm_api.enums.bandpass]` -  pointings associated with bandpasses
- `wavelength_regime: list[float]` - pointings within a wavelength regime (wavelength_unit required)
- `wavelength_unit: str | gwtm_api.enums.wavelength_units` - valid inputs described above 
- `energy_regime: list[float]` - pointings within an energy regime (energy_unit required)
- `energy_unit: str | gwtm_api.enums.energy_units` - valid inputs described above 
- `frequency_regime: list[float]` - pointings within a frequency regime (energy_unit required)
- `frequency_unit: str | gwtm_api.enums.frequency_units` - valid inputs described above 

In [ ]:
queried_pointings = Pointing.get(
    api_token=MY_API_TOKEN, graceid=TEST_GRACEID, status=gwenums.pointing_status.completed
)